# 問題

- 6-armed Bandits

# アルゴリズム

- 方策勾配

## 損失関数

\begin{equation*}
Loss = - \log \pi A
\end{equation*}

Π : アクションのウェイト
A : アドバンテージ(報酬)


# 実行環境

- CPU: Core i7 @2.80GHz
- Mem: 16GB
- HDD: 4TB
- GPU: GeForce1050Ti 4GB
- OS: Ubuntu16.04LTS
- Con: Docker CE/tensorflow1.4.0-gpu/Python3.5.2

In [17]:
import tensorflow as tf
import numpy as np

In [18]:
# 問題

# アームの閾値 値が低いほどあたりが出やすい
bandits = [0.2, -0.5, -0.2, 0, -0.1, 0.2]

# アームの本数
num_bandits = len(bandits)

# アームを引くと結果(1か-1)を返す
def pullBandit(bandit):
    '''
    アームの閾値を渡すとあたり(1)かはずれ(-1)かを返す
    '''
    result = np.random.randn(1)
    if result > bandit:
        return 1
    else:
        return -1

In [19]:
# TensorFlow 計算グラフ

# 計算グラフをリセット
tf.reset_default_graph()

# 重み
weights = tf.Variable(tf.ones([num_bandits]))

# 何番目のアームを引くか
chosen_action = tf.argmax(weights, 0)

# 報酬を格納するための変数
reward_holder = tf.placeholder(shape=[1], dtype=tf.float32)

# アクションを保持するための変数
action_holder = tf.placeholder(shape=[1], dtype=tf.int32)

# アクションのウェイトを取り出す変数　重みから現在選択しているアクションに入っているウェイトを取り出す
responsible_weight = tf.slice(weights, action_holder, [1])

# 損失関数
loss = -(tf.log(responsible_weight)*reward_holder)

# 最適化
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)

# モデル更新
update = optimizer.minimize(loss)

In [21]:
# トレーニング

## パラメータ
# 試行回数
total_episodes = 1000
# 累積報酬
total_reward = np.zeros(num_bandits)
# 閾値 ε
e = 0.1

# TensorFlow 変数の初期化
init = tf.global_variables_initializer()

with tf.Session() as sess:
    # 変数の初期化
    sess.run(init)
    # カウンタ
    i = 0;
    while i < total_episodes:
        # 閾値より乱数が小さい場合
        if np.random.rand(1) < e:
            # ランダムにアクションを決定する
            action = np.random.randint(num_bandits)
        else:
            #print("chosen_action")
            action = sess.run(chosen_action)
        # 報酬を計算
        #print(action)
        reward = pullBandit(bandits[action])
        
        # モデルを更新、ウェイトの取り出し
        _,resp,ww = sess.run([update, responsible_weight, weights], feed_dict={
            reward_holder: [reward], 
            action_holder: [action]})
        
        # 累積報酬
        total_reward[action] += reward
        # 
        if i % 50 == 0:
            print("Reward list: " + str(total_reward))
        i += 1

print("Best arc on agent is " + str(np.argmax(ww) + 1) + " st/nd/rd/th arm!")

Reward list: [-1.  0.  0.  0.  0.  0.]
Reward list: [ -1.  23.   0.   0.   1.   0.]
Reward list: [ -1.  53.   0.   0.   2.  -1.]
Reward list: [ -1.  78.   3.   0.   2.  -1.]
Reward list: [ -1.  86.   3.   0.   3.  -2.]
Reward list: [  -2.  102.    2.    2.    3.   -2.]
Reward list: [  -2.  118.    1.    2.    3.   -1.]
Reward list: [  -3.  127.    1.    2.    5.   -1.]
Reward list: [  -2.  151.    0.    2.    4.   -2.]
Reward list: [  -1.  172.   -2.    1.    5.   -2.]
Reward list: [  -1.  181.   -1.    2.    5.   -3.]
Reward list: [  -3.  195.   -1.    0.    5.   -3.]
Reward list: [  -2.  200.   -4.    0.    6.   -3.]
Reward list: [  -2.  204.   -4.    0.    6.   -3.]
Reward list: [  -3.  209.   -3.    1.    6.   -5.]
Reward list: [  -3.  219.   -4.    1.    3.   -5.]
Reward list: [   0.  230.   -5.    1.    4.   -5.]
Reward list: [   1.  250.   -7.    0.    4.   -5.]
Reward list: [   1.  288.   -7.    0.    4.   -5.]
Reward list: [   1.  304.   -7.    0.    4.   -5.]
Best arc on agen